In [1]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('./src')
from data_loader_df import *
import sys
import numpy
import pandas as pd
from sklearn import svm

In [2]:
train_file_path = "data/adult.data"
test_file_path = "data/adult.test"

train, val, test = load_all_data(train_file_path, test_file_path, valid_rate=0.1, is_df=True, norm=True, one_hot=False)
train_X, train_Y, val_X, val_Y, test_X, test_Y = load_all_data(train_file_path, test_file_path, valid_rate=0.1, is_df=False, norm=True, one_hot=True)

In [4]:
train

,age,work,fnlwgt,edu,edunum,mstatus,occ,relation,race,sex,cgain,closs,hpw,nation,income
24337,1.792267,Local-gov,-1.154262,HS-grad,-0.443917,Married-civ-spouse,Transport-moving,Husband,White,Male,-0.147976,-0.218112,-0.079887,United-States,<=50K
17049,0.879813,Private,0.248813,HS-grad,-0.443917,Married-civ-spouse,Craft-repair,Husband,White,Male,-0.147976,-0.218112,-0.079887,United-States,>50K
21016,-0.184716,Private,-0.041686,HS-grad,-0.443917,Married-civ-spouse,Adm-clerical,Husband,White,Male,-0.147976,-0.218112,0.754722,United-States,>50K
2790,1.944343,Private,0.647885,Some-college,-0.050640,Widowed,Adm-clerical,Not-in-family,White,Female,-0.147976,-0.218112,-0.079887,Cuba,<=50K
13511,-0.793019,Self-emp-not-inc,-1.428567,Assoc-voc,0.342636,Married-civ-spouse,Farming-fishing,Husband,White,Male,-0.147976,-0.218112,1.589331,United-States,<=50K
29288,-1.477359,Private,-0.845366,HS-grad,-0.443917,Never-married,Prof-specialty,Not-in-family,White,Female,-0.147976,-0.218112,-0.079887,United-States,<=50K
2161,-1.325283,Private,-1.226723,Some-college,-0.050640,Never-married,Other-service,Not-in-family,White,Female,-0.147976,-0.218112,-0.330270,United-States,<=50K
12546,0.043397,Private,-0.724826,Bachelors,1.129190,Married-civ-spouse,Exec-managerial,Husband,White,Male,-0.147976,-0.218112,0.337418,United-States,>50K
210,0.423586,Local-gov,-0.274028,HS-grad,-0.443917,Married-civ-spouse,Transport-moving,Husband,Black,Male,-0.147976,-0.218112,-0.079887,United-States,<=50K
20261,2.096419,Private,0.234367,HS-grad,-0.443917,Widowed,Adm-clerical,Not-in-family,White,Female,-0.147976,-0.218112,-2.333331,United-States,<=50K


In [21]:
train.shape

(27158, 15)

In [11]:
np.mean(test[COLS_TO_NORM].values, axis=0)

array([ 0.02578145, -0.0016857 , -0.00629872,  0.00702543,  0.00287755,
       -0.00046595])

In [12]:
np.var(test[COLS_TO_NORM].values, axis=0)

array([1.03511015, 1.00622301, 1.01254746, 1.13583541, 1.01667956,
       1.0135284 ])

In [44]:
cm[0][0] # true negative

11346

In [45]:
cm[1][1] # true positive

551

In [46]:
cm[1][0] # false negative

3149

In [47]:
cm[0][1] # false positive

14

In [77]:
from sklearn.metrics import confusion_matrix
def visualize_confusion_matrix(Y_true, Y_pred):
    cm = confusion_matrix(Y_true, Y_pred).astype(float)
    num_positive = np.sum(Y_true)
    num_negative = len(Y_true) - num_positive
    
    cm[0][0] /= num_negative # true negative rete
    cm[0][1] /= num_negative # false positive rate 
    cm[1][0] /= num_positive # false negative rate
    cm[1][1] /= num_positive # true negative rate
    
    df = pd.DataFrame(
        cm,
        index=["<= 50K", "> 50K"],
        columns=["<= 50K", "> 50K"]
    )
    
    df.index.names = ["Actual"]
    df.columns.names = ["Predicted"]
    return df

In [3]:
df = visualize_confusion_matrix(test_Y, test_Y)

In [86]:
df

Predicted,<= 50K,> 50K
Actual,,
<= 50K,0.998768,0.001232
> 50K,0.851081,0.148919


In [70]:
from sklearn import metrics
import matplotlib.pyplot as plt
def calculate_auc_score(Y_true, Y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(Y_true, Y_pred, pos_label=1)
    print(fpr, tpr, thresholds)
    plt.plot(fpr, tpr, label='RF')
    return metrics.auc(fpr, tpr)

In [3]:
auc = calculate_auc_score(test_Y, test_Y_hat)

NameError: name 'calculate_auc_score' is not defined

In [84]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score
print("roc", roc_auc_score(test_Y, test_Y_hat))
print("acc", accuracy_score(test_Y, test_Y_hat))
print("pre", precision_score(test_Y, test_Y_hat))

roc 0.5738432622763608
acc 0.7899734395750332
pre 0.9752212389380531


In [ ]:
cm[0][1]

In [ ]:
test_Y

In [ ]:
test_Y_hat

In [ ]:
np.sum(test_Y_hat == test_Y) / len(test_Y)

In [ ]:
len(test_Y)

In [ ]:
np.sum(test_Y)

In [ ]:
np.sum(train_Y)

In [ ]:
len(train_Y)

In [ ]:
np.sum(val_Y)

In [ ]:
test_X

In [ ]:
train